In [19]:
!pip install pysastrawi
!pip install lazypredict

In [5]:
import pandas as pd

In [6]:
df = pd.read_excel("data_new.xlsx")
df.head()

,Unnamed: 0,Mengikuti MSIB pada Batch ke,Mitra saat mengikuti Studi Independen,Jalur Pembelajaran atau Kompetensi yang diambil saat mengikuti Studi Independen,Pengalaman,Kategori,Deskripsi
0,0,Bacth ke-5,Bangkit Academy - Dicoding Indonesia,Cloud Computing,Baik,Administrasi,Baik
1,1,Bacth ke-5,Bangkit Academy - Dicoding Indonesia,Cloud Computing,Baik,Proses Belajar-Mengajar,Baik
2,2,Bacth ke-5,Bangkit Academy - Dicoding Indonesia,Cloud Computing,Baik,Capstone Project,"Cukup Struggle di awal, tapi lancar"
3,3,Bacth ke-6,PT. Semesta Integrasi Digital,UI/UX,Baik,Administrasi,aman terkendali
4,4,Bacth ke-6,PT. Semesta Integrasi Digital,UI/UX,Baik,Proses Belajar-Mengajar,terjun langsung dan merasakan menjadi karyawan...


### Data Cleaning

In [10]:
import string
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk import download
from spacy import load

In [11]:
download('stopwords')

def cleaning(corpus: pd.Series):

    list_stopwords = stopwords.words('indonesian')
    stemmer = StemmerFactory().create_stemmer()
    nlp = load("en_core_web_sm")

    corpus = corpus.apply(lambda x: x.lower())\
                   .apply(lambda x: re.findall(r"[a-z]+", x))\
                   .apply(lambda x: [i for i in x if i not in list_stopwords])\
                   .apply(lambda x: [stemmer.stem(i) for i in x])\
                   .apply(lambda x: nlp(" ".join(x)))\
                   .apply(lambda x: [i.lemma_ for i in x])\
                   .apply(lambda x: " ".join(x))

    return corpus

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
df['Deskripsi Bersih'] = cleaning(df['Deskripsi'])
data = df[['Pengalaman', 'Kategori', 'Deskripsi Bersih']]

In [13]:
df.head()

,Unnamed: 0,Mengikuti MSIB pada Batch ke,Mitra saat mengikuti Studi Independen,Jalur Pembelajaran atau Kompetensi yang diambil saat mengikuti Studi Independen,Pengalaman,Kategori,Deskripsi,Deskripsi Bersih
0,0,Bacth ke-5,Bangkit Academy - Dicoding Indonesia,Cloud Computing,Baik,Administrasi,Baik,
1,1,Bacth ke-5,Bangkit Academy - Dicoding Indonesia,Cloud Computing,Baik,Proses Belajar-Mengajar,Baik,
2,2,Bacth ke-5,Bangkit Academy - Dicoding Indonesia,Cloud Computing,Baik,Capstone Project,"Cukup Struggle di awal, tapi lancar",struggle lancar
3,3,Bacth ke-6,PT. Semesta Integrasi Digital,UI/UX,Baik,Administrasi,aman terkendali,aman kendali
4,4,Bacth ke-6,PT. Semesta Integrasi Digital,UI/UX,Baik,Proses Belajar-Mengajar,terjun langsung dan merasakan menjadi karyawan...,terjun langsung rasa karyawan alam ajar dapat


### Data Preprocessing

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split as tts

In [15]:
vect = TfidfVectorizer()
enc1 = LabelEncoder()
enc2 = LabelEncoder()

x = vect.fit_transform(data['Deskripsi Bersih'])
y1 = enc1.fit_transform(data['Pengalaman'])
y2 = enc2.fit_transform(data['Kategori'])

In [23]:
x_train, x_test, y1_train, y1_test = tts(x, y1,
                                         test_size=0.2,
                                         random_state=20)
x_train, x_test, y2_train, y2_test = tts(x, y2,
                                         test_size=0.2,
                                         random_state=20)

In [20]:
from lazypredict.Supervised import LazyClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [27]:
# Initialize LazyClassifier
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

# Fit on training data and evaluate on test data
models, predictions = clf.fit(x_train.toarray(), x_test.toarray(), y1_train, y1_test)

# Print the results
print(models)

100%|██████████| 29/29 [00:01<00:00, 14.85it/s]

[LightGBM] [Info] Number of positive: 77, number of negative: 100
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000055 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 66
[LightGBM] [Info] Number of data points in the train set: 177, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.435028 -> initscore=-0.261365
[LightGBM] [Info] Start training from score -0.261365
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[L

In [28]:
predictions

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
NearestCentroid,0.84,0.81,0.81,0.84,0.03
ExtraTreesClassifier,0.84,0.80,0.80,0.84,0.16
BernoulliNB,0.84,0.78,0.78,0.83,0.02
NuSVC,0.82,0.78,0.78,0.82,0.04
DecisionTreeClassifier,0.80,0.76,0.76,0.79,0.04
ExtraTreeClassifier,0.82,0.75,0.75,0.80,0.02
LinearSVC,0.76,0.74,0.74,0.76,0.11
PassiveAggressiveClassifier,0.76,0.74,0.74,0.76,0.04
BaggingClassifier,0.80,0.73,0.73,0.78,0.06


In [25]:
# Initialize LazyClassifier
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

# Fit on training data and evaluate on test data
models, predictions = clf.fit(x_train.toarray(), x_test.toarray(), y2_train, y2_test)

# Print the results
print(models)

100%|██████████| 29/29 [00:01<00:00, 14.66it/s]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000036 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 66
[LightGBM] [Info] Number of data points in the train set: 177, number of used features: 6
[LightGBM] [Info] Start training from score -1.049015
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.150798
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

In [29]:
import joblib

In [31]:
for model_name, model in clf.models.items():

    if model_name == 'NearestCentroid':

        filename = f'{model_name}.pkl'
        joblib.dump(model, filename)
        print(f'{model_name} saved as {filename}')

NearestCentroid saved as NearestCentroid.pkl


In [38]:
x_train = x_train.toarray()
x_test = x_test.toarray()

In [42]:
model1 = joblib.load('NearestCentroid.joblib')
model1.predict(x_test)

array([1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       1])

In [44]:
pred = model1.predict(x.toarray())
acc = sum([1 for i in pred if i in y1]) / len(pred)
print(acc)

1.0


In [50]:
len(pred)

222

In [58]:
df['Hasil Prediksi'] = pred
df['Hasil Prediksi'] = df['Hasil Prediksi'].apply(lambda x: 1 if x == 0 else 0)\
                                           #.replace({0:'Buruk', 1:'Baik'})
df.head()

,Unnamed: 0,Mengikuti MSIB pada Batch ke,Mitra saat mengikuti Studi Independen,Jalur Pembelajaran atau Kompetensi yang diambil saat mengikuti Studi Independen,Pengalaman,Kategori,Deskripsi,Deskripsi Bersih,Hasil Prediksi
0,0,Bacth ke-5,Bangkit Academy - Dicoding Indonesia,Cloud Computing,Baik,Administrasi,Baik,,1
1,1,Bacth ke-5,Bangkit Academy - Dicoding Indonesia,Cloud Computing,Baik,Proses Belajar-Mengajar,Baik,,1
2,2,Bacth ke-5,Bangkit Academy - Dicoding Indonesia,Cloud Computing,Baik,Capstone Project,"Cukup Struggle di awal, tapi lancar",struggle lancar,1
3,3,Bacth ke-6,PT. Semesta Integrasi Digital,UI/UX,Baik,Administrasi,aman terkendali,aman kendali,1
4,4,Bacth ke-6,PT. Semesta Integrasi Digital,UI/UX,Baik,Proses Belajar-Mengajar,terjun langsung dan merasakan menjadi karyawan...,terjun langsung rasa karyawan alam ajar dapat,1


In [59]:
df.to_excel("df_final.xlsx")